In [2]:
from vllm import LLM, SamplingParams

llm = LLM(model="facebook/opt-6.7b")  # 실행할 모델 지정
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

outputs = llm.generate("vLLM은 무엇인가요?", sampling_params)
print(outputs[0].text)  # 생성된 텍스트 출력


ModuleNotFoundError: No module named 'vllm'